# How do socioeconomic factors affect crime rates in Seattle?
## Chantria Im, Ellie Velez, and Nawaf Osman

In [1]:
%%capture
%%script echo skipping # Skip this code cell

# Install packages
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install requests

In [2]:
# Import libraries
import pandas as pd
import requests
import re
import time

In [3]:
# Load datasets
crime_df = pd.read_csv('./data/SPD_Crime_Data__2008-Present_20240226.csv')
poverty_df = pd.read_csv('./data/Poverty_and_Near_Poverty_Map_Full_Data_data.csv', 
                         converters={'Census Tract': str.strip})
income_df = pd.read_csv('./data/ACSST5Y2018.S1903-Data.csv')

In [4]:
# Remove rows with missing Longitude and Latitude values
crime_df = crime_df[(crime_df['Longitude'] != 0) & (crime_df['Latitude'] != 0)]

# Filter for only 2018 entries
crime_df['Report DateTime'] = pd.to_datetime(crime_df['Report DateTime'], format='%m/%d/%Y %I:%M:%S %p')
crime_df = crime_df[crime_df['Report DateTime'].dt.year == 2018]

# Add Census Tract column
crime_df['Census Tract'] = pd.Series(dtype='string')

In [5]:
%%capture
%%script echo skipping # Skip this code cell

# Fill Census Tract column with corresponding census tract numbers
i = 0
while i < len(crime_df):
    # Make request to Census Geocoder API to get row's census tract data based on row's longtitude and latitude values
    payload = {'benchmark': 'Public_AR_Current', 'vintage': 'ACS2018_Current', 'x': crime_df['Longitude'].iloc[i], 
               'y': crime_df['Latitude'].iloc[i], 'format': 'json', 'layers': 'Census Tracts'}
    try:
        r = requests.get(f'https://geocoding.geo.census.gov/geocoder/geographies/coordinates', params=payload)
    except requests.Timeout:
        time.sleep(300)
        continue
    # Extract the census tract number from the response and store it in the Census Tract column
    crime_df['Census Tract'].iloc[i] = re.findall('\d+\.\d+|\d+', 
                                                  r.json()['result']['geographies']['Census Tracts'][0]['NAME'])[0]
    i += 1

# Backup dataframe to disk 
crime_df.to_csv('./data/crime.csv')

In [6]:
pd.set_option('display.max_colwidth', None)

# Cache the dataframe
# %store crime_df
# Reload the dataframe from cache rather than recomputing (5+ hours)
%store -r crime_df

In [7]:
# Drop empty columns
poverty_df = poverty_df.drop(['% of Population Under 100% Poverty Line', '% of Population Under 200% Poverty Line', 
                              'Share Below Selected % of Poverty Level'], axis=1)

# Group data by census tract and calculate total poverty population for levels
poverty_by_tract = poverty_df[poverty_df['Name'] != 'Total'].groupby('Census Tract').agg(
    {'Population Under 100% Poverty Level': 'sum', 'Population Under 200% Poverty Level': 'sum'})

# For each census tract fill designated 'Total' row with total poverty populations for levels
poverty_df.loc[
    poverty_df['Name'] == 'Total', 
    'Population Under 100% Poverty Level'] = poverty_df.loc[
    poverty_df['Name'] == 'Total', 'Census Tract'].map(poverty_by_tract['Population Under 100% Poverty Level']).values
poverty_df.loc[
    poverty_df['Name'] == 'Total', 
    'Population Under 200% Poverty Level'] = poverty_df.loc[
    poverty_df['Name'] == 'Total', 'Census Tract'].map(poverty_by_tract['Population Under 200% Poverty Level']).values

# Filter for only designated 'Total' rows
poverty_df = poverty_df[poverty_df['Name'] == 'Total']

In [8]:
# Remove rows with missing median income values
income_df = income_df[pd.to_numeric(income_df['S1903_C03_001E'], errors='coerce').notna()] # 53.02 University District Missing

# Extract and set census tract numbers
for i in range(len(income_df)):
    income_df.iat[i, income_df.columns.get_loc('NAME')] = re.findall('\d+\.\d+|\d+', income_df['NAME'].iloc[i])[0]
    
# Edit column names
income_df = income_df.rename(columns={'NAME': 'Census Tract', 'S1903_C03_001E': 'Median Income'})

# Set median income column to numeric type
income_df['Median Income'] = pd.to_numeric(income_df['Median Income'])

In [9]:
# Drop un-needed columns
crime_df = crime_df.filter(['Census Tract', 'Offense Start DateTime', 'Offense End DateTime', 'Report DateTime', 
                            'Offense Parent Group'])
poverty_df = poverty_df.filter(['Census Tract', 'Population Under 100% Poverty Level', 
                                'Population Under 200% Poverty Level', 'Total Population'])
income_df = income_df.filter(['Census Tract', 'Median Income'])

In [10]:
# Merge datasets
merged_df = crime_df.merge(poverty_df, on='Census Tract')
merged_df = merged_df.merge(income_df, on='Census Tract')

In [21]:
# Calculate total crime offenses  by census tract
grouped_df = merged_df.groupby('Census Tract').agg({
    'Population Under 100% Poverty Level': 'first',
    'Population Under 200% Poverty Level': 'first',
    'Total Population': 'first',
    'Median Income': 'first',
    'Offense Parent Group': 'count'
})
grouped_df = grouped_df.rename(columns={'Offense Parent Group': 'Total Offenses'})

# Calculate crime rate per census tract
# Crime rate is calculated by: total crime offenses / total population * 100,000
grouped_df['Crime Rate per 100,000'] = (grouped_df['Total Offenses'] / grouped_df['Total Population']) * 100000
grouped_df['Crime Rate per 100,000'] = grouped_df['Crime Rate per 100,000'].round()

grouped_df

,Population Under 100% Poverty Level,Population Under 200% Poverty Level,Total Population,Median Income,Total Offenses,"Crime Rate per 100,000"
Census Tract,,,,,,
1,1159,2741,7978,50948,848,10629.0
10,311,576,2055,83295,132,6423.0
100.01,587,1674,4448,63891,368,8273.0
100.02,366,1316,4910,74953,283,5764.0
101,1248,2515,8763,87188,842,9609.0
...,...,...,...,...,...,...
96,163,634,5564,102569,276,4960.0
97.01,221,715,6180,110368,251,4061.0
97.02,276,462,5596,120189,166,2966.0


In [20]:
# Calculate crime rate per census tract by offense type
offenses_df = merged_df.groupby(['Census Tract','Population Under 100% Poverty Level', 
                                 'Population Under 200% Poverty Level', 'Total Population', 
                                 'Median Income', 'Offense Parent Group']).size().reset_index(name='Offense Count')
offenses_df['Crime Rate per 100,000'] = (offenses_df['Offense Count'] / offenses_df['Total Population']) * 100000
offenses_df['Crime Rate per 100,000'] = offenses_df['Crime Rate per 100,000'].round()

offenses_df

,Census Tract,Population Under 100% Poverty Level,Population Under 200% Poverty Level,Total Population,Median Income,Offense Parent Group,Offense Count,"Crime Rate per 100,000"
0,1,1159,2741,7978,50948,ARSON,1,13.0
1,1,1159,2741,7978,50948,ASSAULT OFFENSES,137,1717.0
2,1,1159,2741,7978,50948,BAD CHECKS,15,188.0
3,1,1159,2741,7978,50948,BURGLARY/BREAKING&ENTERING,103,1291.0
4,1,1159,2741,7978,50948,COUNTERFEITING/FORGERY,3,38.0
...,...,...,...,...,...,...,...,...
2041,99,403,1035,6180,81066,PORNOGRAPHY/OBSCENE MATERIAL,1,16.0
2042,99,403,1035,6180,81066,ROBBERY,5,81.0
2043,99,403,1035,6180,81066,STOLEN PROPERTY OFFENSES,4,65.0
2044,99,403,1035,6180,81066,TRESPASS OF REAL PROPERTY,29,469.0


In [22]:
# Remove rows with missing Offense Start and End DateTime values
policing_df = merged_df[merged_df['Offense Start DateTime'].notna() & merged_df['Offense End DateTime'].notna()].copy()

# Change column types to DateTime
policing_df['Offense Start DateTime'] = pd.to_datetime(policing_df['Offense Start DateTime'], 
                                                       format='%m/%d/%Y %I:%M:%S %p')
policing_df['Offense End DateTime'] = pd.to_datetime(policing_df['Offense End DateTime'], 
                                                     format='%m/%d/%Y %I:%M:%S %p')

policing_df

,Census Tract,Offense Start DateTime,Offense End DateTime,Report DateTime,Offense Parent Group,Population Under 100% Poverty Level,Population Under 200% Poverty Level,Total Population,Median Income
2,56,2018-12-24 18:00:00,2018-12-25 11:00:00,2018-12-25 14:14:00,LARCENY-THEFT,170,353,6954,170031
4,56,2018-12-18 16:00:00,2018-12-20 18:00:00,2018-12-23 18:20:00,LARCENY-THEFT,170,353,6954,170031
5,56,2018-11-06 00:00:00,2018-12-04 00:00:00,2018-12-17 13:03:00,LARCENY-THEFT,170,353,6954,170031
6,56,2018-12-12 06:10:00,2018-12-17 11:10:00,2018-12-17 11:50:00,DESTRUCTION/DAMAGE/VANDALISM OF PROPERTY,170,353,6954,170031
7,56,2018-12-14 12:00:00,2018-12-15 10:00:00,2018-12-16 12:33:00,DESTRUCTION/DAMAGE/VANDALISM OF PROPERTY,170,353,6954,170031
...,...,...,...,...,...,...,...,...,...
72424,213,2018-07-05 03:10:00,2018-07-05 04:52:00,2018-07-05 04:52:00,LARCENY-THEFT,294,595,3941,80227
72425,213,2018-05-31 19:10:00,2018-05-31 20:00:00,2018-06-01 19:52:00,LARCENY-THEFT,294,595,3941,80227
72429,213,2018-05-11 13:00:00,2018-05-11 16:30:00,2018-05-14 09:59:00,DESTRUCTION/DAMAGE/VANDALISM OF PROPERTY,294,595,3941,80227
72430,213,2018-04-22 02:00:00,2018-04-22 07:45:00,2018-04-22 14:04:00,FRAUD OFFENSES,294,595,3941,80227
